<a href="https://colab.research.google.com/github/BenjamintsKang/AIFFEL_Quest/blob/master/%5BLangChainKR_2024Q2%5DNative_RAG_to_Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [LangChainKR 2024Q2]Native RAG to Advanced RAG 톺아보기
- 작성자 : 백혜림(rimiyeyo@gmail.com)
- Reference : https://github.com/langchain-ai


In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.0 MB/s eta 0:00:00


# Colab Enviornment

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# 1.Native RAG
    - 문서 불러오기
    - 문서 분할하기
    - 백터 데이터베이스에 임베딩 저장하기
    - 검색기 만들기
    - 프롬프트 만들기
    - 체인 만들기
    - 질문하고 답변얻기

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## Indexing

In [ ]:
loader = WebBaseLoader("https://aifactory.space/task/4239/overview", encoding="utf-8")

docs = loader.load()

### Split

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

9

### Embedding

In [ ]:
open_ai_embedding = OpenAIEmbeddings()

### Retrieval

In [ ]:

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=open_ai_embedding)

retriever = vectorstore.as_retriever()

## Generation

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature= 0,
    max_tokens = 2048,
    model_name = "gpt-3.5-turbo",
)

### Post-processing

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## 체인에 질문하고 답변받기

In [ ]:
rag_chain.invoke("랭체인 연사자가 누구야?")

'김태영, 유현아입니다.'

# 2.MultiQueryRetriever  
거리 기반 벡터DB 검색은 쿼리를 임베딩하고 거리 기준으로 유사한 문서를 찾습니다. 쿼리의 의미가 불분명하면 임베딩이 데이터의 의미파악을 제대로 하지못하는 단점이 있습니다.

이러한 단점을 해결하기 위해 MultiQueryRetriever는 LLM을 사용해 입력된 쿼리에 대해 서로 다른 관점에서 여러 쿼리를 생성함으로서 프롬프트 튜닝을 자동화합니다.

In [ ]:
# load
loader = WebBaseLoader("https://aifactory.space/task/4239/overview", encoding="utf-8")
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

# VectorDB
openai_embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=openai_embedding)

## LangChain MultiQueryRetriever 사용

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "랭체인 연사자는 누구인가요?"
llm = ChatOpenAI(
    temperature= 0,
    max_tokens = 2048,
    model_name = "gpt-3.5-turbo",
)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [ ]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. 랭체인 연사자의 실명은 무엇인가요?', '2. 랭체인 연사자의 역할은 무엇인가요?', '3. 랭체인 연사자의 경력과 업적은 무엇인가요?']


1

## 직접 프롬프트로 MultiQuery생성하기

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: 다른 시각으로 보는 템플릿을 작성하자!
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
generate_queries.invoke("랭체인 연사자는 누구인가요?")

['1. 랭체인 연사자의 신분은 무엇인가요?',
 '2. 랭체인 연사자의 역할은 무엇인가요?',
 '3. 랭체인 연사자의 업무 내용은 무엇인가요?',
 '4. 랭체인 연사자의 역사는 어떻게 되나요?',
 '5. 랭체인 연사자의 중요성은 무엇인가요?']

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ retrieved docs들을 하나의 문서로 합치는 과정 """
    # 리스트의 리스트를 쭉 펼치고, 각 문서를 문자열로 변환한다.
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # 유니크한 문서를 얻는 과정
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "랭체인 연사자는 누구인가요?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


1

In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'랭체인 연사자는 백혜림, 테디노트(이경록), 김태영 입니다.'

# 3.Decomposition
일부 복잡한 질문의 경우 한번의 검색 단계로 해결되지 않을 수도 있습니다. 그래서 문제를 순차적으로 (첫 번째 답변 + 검색을 사용해서 두 번쨰 답변에 답변) 혹은 병렬로 (각 답변을 최종답변으로 통합)해결할수 있습니다.

- Least-to-Most Prompting 혹은 IR-CoT을 활용할 수 있습니다

In [ ]:
from langchain.prompts import ChatPromptTemplate

# Decomposition (Least to mode prompt사용해도 무방!)
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatOpenAI(temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "랭체인(LangChain)과 랭그래프(LangGraph)는 무엇인가요?"
questions = generate_queries_decomposition.invoke({"question":question})

In [ ]:
questions

['1. 랭체인(LangChain)이란 무엇인가요?',
 '2. 랭그래프(LangGraph)는 어떤 기술이며 어떻게 작동하나요?',
 '3. 랭체인(LangChain)과 랭그래프(LangGraph)의 차이점은 무엇인가요?']

# 4.RAG-Fusion  
여러개 쿼리를 생성해서 re-rank search결과로 Reciprocal Rank Fusion(RRF)을 사용합니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """ RRF공식에서 사용되는 파라미터 k로 여러 순위 문서들을 처리하는 Reciprocal_rank_fusion"""

    # 각 유니크 문서에 대한 fused score를 저장하기 위해 딕셔너리 초기화
    fused_scores = {}

    # 각 ranked 문서들을 반복해라
    for docs in results:
        # 리스트에서 각 문서를 rank로 반복 (리스트의 위치에서)
        for rank, doc in enumerate(docs):
            # 문서를 문자열 형식으로 변환하여 키로 사용
            doc_str = dumps(doc)
            # 문서가 아직 fused_scores 딕셔너리에 없으면 초기 점수 0으로 추가
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 문서의 현재 점수가 있다면 검색하기
            previous_score = fused_scores[doc_str]
            # RRF 공식: 1 / (rank + k) 을 사용하여 문서의 점수를 업데이트
            fused_scores[doc_str] += 1 / (rank + k)

    # 최종 재순위 결과를 얻기 위해 문서들을 fused_scores를 기준으로 내림차순으로 정렬.
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 문서와 fused_scores를 각각 포함하는 튜플의 리스트로 재순위된 결과를 반환합니다
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)

1

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'랭체인(LangChain)은 Native RAG부터 Advanced RAG까지 톺아보는 것이며, 랭그래프(LangGraph)는 초보자도 할 수 있는 고급RAG로 다중 에이전트와 LangGraph를 제작하는 것을 의미합니다.'

# 5.Self Query

In [ ]:
%pip install --upgrade --quiet  lark langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypr

In [ ]:
from langchain.schema import Document
embeddings = OpenAIEmbeddings()

In [ ]:
docs = [
    Document(
        page_content="시간을 아껴주는 말하는 앵무새 :: 유튭 정리에서 데이터 시각화까지",
        metadata={"name":"전미정", "year": 2023},
    ),
    Document(
        page_content="Whisper보다 6배빠른 ditil-Whisper로 오디오데이터에서 RAG 수행기",
        metadata={"name":"백혜림", "year": 2023},
    ),
    Document(
        page_content="Native RAG부터 Advanced RAG 톺아보기",
        metadata={"name":"백혜림", "year": 2024},
    ),
    Document(
        page_content="초보자도 할 수 있는 고급RAG : 다중 에이전트와 LangGraph 제작",
        metadata={"name":"이경록", "year": 2024},
    ),
    Document(
        page_content="한국어 오픈액세스 LM의 시각과 그 이후",
        metadata={"name":"이준범", "year": 2023},
    ),
    Document(
        page_content="LLM으로 LLM을 해킹했습니다.",
        metadata={"name":"백승윤", "year": 2023},
    ),
    Document(
        page_content="LCEL 치트시트",
        metadata={"name":"김태영", "year": 2024},
    ),
]
vectorstore = Chroma.from_documents(docs, embeddings)

In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="name",
        description="랭체인 연사자 이름",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="랭체인발표 진행한 년도",
        type="integer",
    )
]
document_content_description = "랭체인코리아 발표자 목록"



In [ ]:
llm = ChatOpenAI(temperature=0)

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [ ]:
# This example only specifies a relevant query
retriever.invoke("2024년에 발표한 랭체인 발표주제는 무엇인가요?")

[Document(page_content='LCEL 치트시트', metadata={'name': '김태영', 'year': 2024}),
 Document(page_content='초보자도 할 수 있는 고급RAG : 다중 에이전트와 LangGraph 제작', metadata={'name': '이경록', 'year': 2024}),
 Document(page_content='Native RAG부터 Advanced RAG 톺아보기', metadata={'name': '백혜림', 'year': 2024})]